In [17]:
import os
import cv2
import math
import pickle
import numpy as np

import sys

# sys.path.append("../arcface/")

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split


from tensorflow import keras
import tensorflow as tf

from arcface.dataset import build_dataset
from arcface.losses import ArcLoss
from arcface.network import ArcLayer, L2Normalization, resnet101
from arcface.training_supervisor import TrainingSupervisor

from tensorflow.keras.applications.efficientnet import EfficientNetB3

import IPython.display as display
from glob import glob
from tqdm import tqdm
from abc import ABC, abstractmethod

import albumentations as A
from matplotlib import pyplot as plt



In [2]:
### For cpu only
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
tf.__version__

'2.4.1'

# TODO


1. hrnet (Done)
2. resnet ()
3. efficienthrnet
4. efficientnet


# Augment

In [29]:
def make_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)


def plot(images):
    from matplotlib import pyplot as plt
    n=len(images)
    fig, axes = plt.subplots(ncols=n, figsize=(16, 5))
    ax = axes.ravel()
    for i in range(1,n+1):
        ax[i-1] = plt.subplot(1, n, i)
        ax[i-1].imshow(images[i-1])
    
    plt.show()

# reduce noise in the image
def reduce_noise(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    noise = cv2.fastNlMeansDenoising(gray)
    return cv2.cvtColor(noise, cv2.COLOR_GRAY2BGR)

# histogram equalization
def equalize_hist(img, kernel=np.ones((7,7),np.uint8)):
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

# invert a binary image
def invert(img):
    return cv2.bitwise_not(img)

def inverted(img):
    return (255-img)

# gray
def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# skeletonize the image
def skel(gray):
    img = gray.copy()
    skel = img.copy()
    skel[:,:] = 0
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5))

    while cv2.countNonZero(img) != 0:
        eroded = cv2.morphologyEx(img, cv2.MORPH_ERODE, kernel)
        temp = cv2.morphologyEx(eroded, cv2.MORPH_DILATE, kernel)
        temp = cv2.subtract(img, temp)
        skel = cv2.bitwise_or(skel, temp)
        img[:,:] = eroded[:,:]
    return skel

# threshold to make the veins more visible
def thresh(img):
    _, thr = cv2.threshold(img, 5,255, cv2.THRESH_BINARY)
    return thr

def Sift(img, skeleton):
    scale = cv2.SIFT_create()
    kp = scale.detect(skeleton,None)
    img_copy=img.copy()
    img_copy=cv2.drawKeypoints(skeleton,kp,img_copy)
    
    return img_copy

def background_substraction_1(img):
    #equalization
    clahe = cv2.createCLAHE(clipLimit=20.0, tileGridSize=(8,8))
    img= clahe.apply(img)
    CAP2 = img.copy()
    #Filter medianBlur
    CAP3 = cv2.medianBlur(img,5)
    
    return CAP3

def background_substraction_2(img):
    #equalization
    clahe = cv2.createCLAHE(clipLimit=20.0, tileGridSize=(8,8))
    img= clahe.apply(img)
    CAP2 = img.copy()
    #Binarización
    ret, CAP4 = cv2.threshold(CAP2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    CAP4 = 255-CAP4.copy()

    return CAP4


def ridge(img, ridge_filter=cv2.ximgproc.RidgeDetectionFilter_create()):
    ridges = ridge_filter.getRidgeFilteredImage(img)
    return ridges
    
def process_image(imgIn, imgOut=None):
#     print("processing...")
    img = cv2.imread(imgIn)
    kernel = np.ones((7,7),np.uint8)
    noise = reduce_noise(img)
    img_output = equalize_hist(noise, kernel)
    inv = invert(img_output)
    gray_scale = gray(inv)
#     print("skeletonizing...")
    skeleton = skel(gray_scale)
    thr = thresh(skeleton)
    scale = Sift(img, skeleton)

#     cv2.imwrite(imgOut, thr)
#     plot([img, img_output, gray_scale, skeleton, thr, scale])
#     print("done")

    return thr

def resize(img, dim=(128,128)):
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

def proc_maker(list_func, img_path):
    
    img = cv2.imread(img_path)
    
    proc_img = [img]
    for func in list_func:
        proc_img.append(func(proc_img[-1]))
        
#     plot(proc_img)
    
    return proc_img[-1]

def preprocess(procs, dataset_path='datasets/sorted_palmvein_roi/', types='bmp', output_path='datasets/processed/'):
    imagePaths =[]
    [imagePaths.extend(glob(dataset_path+'/*/*.'+image_type)) for image_type in types]
    make_folder(output_path)
    flag=set()
    for img_path in tqdm(imagePaths):
        folder_name = img_path.split('/')[-2]
        name = img_path.split('/')[-1]
        img =  proc_maker(procs, img_path)
        if folder_name in flag:
            cv2.imwrite(output_path+folder_name+'/'+name, img)
        else:
            make_folder(output_path+folder_name)
            cv2.imwrite(output_path+folder_name+'/'+name, img)
            flag.add(folder_name)
            
            
def augment_data(path = 'datasets/sorted_palmvein_roi/',  types=[".bmp"], dataset_path='datasets/augmented/', aug_num=12):
    
    data_augmentation = A.Compose([
            A.Rotate(limit=(0,360), p=1, border_mode=0),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=1),
            # A.ShiftScaleRotate(scale_limit=0.0, shift_limit_x=0, shift_limit_y=0)
        ])
    
    samples =[]
    [samples.extend(glob(path+"*/*."+typ)) for typ in types]
    
    for i, image_path in tqdm(enumerate(samples)):
        
        label = image_path.split('/')[-2]
        
        name = image_path.split('/')[-1]
        filename, _type= name.split(".")
        
        directory = dataset_path+"/"+label
        make_folder(directory)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (128,128))
        cv2.imwrite(directory+"/"+name, image)
        
        for j in range(1, aug_num):
            cv2.imwrite(directory+"/"+f"{filename}_{j}.{_type}", data_augmentation(image=image)["image"])
                      



In [32]:
name='600_unprocessed'
augment_data(path= 'datasets/'+name+"/", types=['png','jpg','jepg','png', 'bmp'], dataset_path='datasets/600+500_aug/', aug_num=9)

name='500_unprocessed'
augment_data(path= 'datasets/'+name+"/", types=['png','jpg','jepg','png', 'bmp'], dataset_path='datasets/600+500_aug/', aug_num=5)

18000it [08:37, 34.81it/s]


In [ ]:
ALL_PROCS = [reduce_noise, equalize_hist, invert, gray, skel, thresh, Sift, background_substraction_1, background_substraction_2, ridge, inverted, resize]
proc=[reduce_noise, equalize_hist, gray, invert, skel, background_substraction_2, invert, resize]

preprocess(proc, dataset_path='datasets/600+500_aug', types=['png','jpg','jepg','png', 'bmp'], output_path='datasets/600+500_processed/')

  4%|██▋                                                                | 8100/198000 [04:16<1:38:54, 32.00it/s]

### TFRecord

In [21]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def create_example(image, label):
    feature = {
        "image": image_feature(image),
        "label": bytes_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

### Backbone Class make (ToDo Make into a separate module)############################
class RecordOperator(ABC):

    def __init__(self, filename):
        # Construct a reader if the user is trying to read the record file.
        self.dataset = None
        self._writer = None
        if tf.io.gfile.exists(filename):
            self.dataset = tf.data.TFRecordDataset(filename)
        else:
            # Construct a writer in case the user want to write something.
            self._writer = tf.io.TFRecordWriter(filename)

        # Set the feature description. This should be provided before trying to
        # parse the record file.
        self.set_feature_description()

    @abstractmethod
    def make_example(self):
        """Returns a tf.train.example from values to be saved."""
        pass

    def write_example(self, tf_example):
        """Create TFRecord example from a data sample."""
        if self._writer is None:
            raise IOError("Record file already exists.")
        else:
            self._writer.write(tf_example.SerializeToString())

    @abstractmethod
    def set_feature_description(self):
        """Set the feature_description to parse TFRecord file."""
        pass

    def parse_dataset(self):
        # Create a dictionary describing the features. This dict should be
        # consistent with the one used while generating the record file.
        if self.dataset is None:
            raise IOError("Dataset file not found.")

        def _parse_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            return tf.io.parse_single_example(example_proto, self.feature_description)

        parsed_dataset = self.dataset.map(_parse_function)
        return parsed_dataset
#########################################################################################

class ImageDataset(RecordOperator):
    """Construct ImageDataset tfrecord files."""

    def make_example(self, image, label):
        """Construct an tf.Example with image data and label.
        Args:
            image_string: encoded image, NOT as numpy array.
            label: the label.
        Returns:
            a tf.Example.
        """
        
        image_string = tf.image.decode_image(image)
        image_shape = image_string.shape
        

        # After getting all the features, time to generate a TensorFlow example.
        feature = {
            'image/height': int64_feature(image_shape[0]),
            'image/width': int64_feature(image_shape[1]),
            'image/depth': int64_feature(image_shape[2]),
            'image/encoded': image_feature(image_string),
            'label': int64_feature(label),
        }

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=feature))

        return tf_example
    
    def set_feature_description(self):
        self.feature_description = {
            "image/height": tf.io.FixedLenFeature([], tf.int64),
            "image/width": tf.io.FixedLenFeature([], tf.int64),
            "image/depth": tf.io.FixedLenFeature([], tf.int64),
            "image/encoded": tf.io.VarLenFeature(tf.float32),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
#         example = tf.io.parse_single_example(example, feature_description)
#         example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
#         return example

def split_tfrecord_creator(split_samples, split_tf_record):
        converter = ImageDataset(split_tf_record)
        total_samples_num = len(split_samples)
        ids = set()

        for i, image_path in enumerate(split_samples):
            image = tf.io.read_file(image_path)
            ids.add(image_path.split('/')[-2])
            label = len(ids)
            tf_example = converter.make_example(image, label)
            # Write the example to file.
            converter.write_example(tf_example)

        print(f"Total num_ids: {len(ids)}, Total num_examples: {total_samples_num}")
        print("All done. Record file is:\n{}".format(split_tf_record))
        

def create_tfrecord(path = 'datasets/sorted_palmvein_roi/',  types=[".bmp"], tf_record='datasets/train.record', test_size=0.2, random_state=42):
    samples =[]
    [samples.extend(glob(path+"*/*."+typ)) for typ in types]
    labels = [image_path.split('/')[-2] for image_path in samples]
    sample_labels = [image_path.split('/')[-2] for image_path in samples]
    samples_train, samples_val = train_test_split(samples, test_size=test_size, random_state=random_state, stratify=sample_labels)
    
    split_tfrecord_creator(split_samples= samples_train, split_tf_record= tf_record+"_train.record")
    split_tfrecord_creator(split_samples= samples_val, split_tf_record= tf_record+"_val.record")
    


In [7]:
name='600+500_processed'
create_tfrecord(path= 'datasets/'+name+"/", types=['png','jpg','jepg','png', 'bmp'], tf_record='datasets/'+name, test_size=0.2, random_state=42)

2022-07-27 20:51:12.953048: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-27 20:51:12.960758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-27 20:51:13.327428: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-27 20:51:13.327656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 computeCapability: 7.5
coreClock: 1.815GHz coreCount: 22 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 178.86GiB/s
2022-07-27 20:51:13.327806: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-27 20:51:13.342503: I tensorflow/stream_executor/platfo

Total num_ids: 1142, Total num_examples: 24721
All done. Record file is:
datasets/processed_merge_train.record
Total num_ids: 1142, Total num_examples: 6181
All done. Record file is:
datasets/processed_merge_val.record


# Training

In [3]:
def train(base_model, name = "hrnetv2", train_files = "datasets/train.record", test_files = None, val_files = None, input_shape = (128, 128, 3),
          num_ids = 600, num_examples = 12000, training_dir = os.getcwd(),
          val_num_ids = 600, val_num_examples = 12000, val_batch_size = 8, val_freq=None,
          frequency = 1000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = 8, export_only = False,
          override = False, epochs = 50, restore_weights_only=False):

    '''
    # Deep neural network training is complicated. The first thing is making
    # sure you have everything ready for training, like datasets, checkpoints,
    # logs, etc. Modify these paths to suit your needs.

    name:str = # What is the model's name?
    
    train_files:str = # Where are the training files?

    test_files:str = # Where are the testing files?

    val_files:str = # Where are the validation files? Set `None` if no files available.

    input_shape:tuple(int) = # What is the shape of the input image?

    embedding_size:int = # What is the size of the embeddings that represent the faces?

    num_ids:int = # How many identities do you have in the training dataset?

    num_examples:int = # How many examples do you have in the training dataset?

    # That should be sufficient for training. However if you want more
    # customization, please keep going.

    training_dir:str = # Where is the training direcotory for checkpoints and logs?

    regularizer = # Any weight regularization?

    frequency:int = # How often do you want to log and save the model, in steps?

    # All sets. Now it's time to build the model. There are two steps in ArcFace
    # training: 1, training with softmax loss; 2, training with arcloss. This
    # means not only different loss functions but also fragmented models.

    base_model:model = # First model is base model which outputs the face embeddings.
    '''
    
    # Where is the exported model going to be saved?
    export_dir = os.path.join(training_dir, 'exported', name)
    
#     data_augmentation = tf.keras.Sequential([
#        tf.keras.layers.RandomRotation(factor=0.05),
#        tf.keras.layers.RandomContrast(factor=0.2),
#        tf.keras.layers.RandomZoom(height_factor=0.1)
#      ])
    
    # Then build the second model for training.
    if softmax:
        print("Building training model with softmax loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  # data_augmentation,
                                  base_model,
                                  keras.layers.Dense(num_ids,
                                                     kernel_regularizer=regularizer),
                                  keras.layers.Softmax()],
                                 name="training_model")
        loss_fun = keras.losses.CategoricalCrossentropy()
    else:
        print("Building training model with ARC loss...")
        model = keras.Sequential([keras.Input(input_shape),
                                  # data_augmentation,
                                  base_model,
                                  L2Normalization(),
                                  ArcLayer(num_ids, regularizer)],
                                 name="training_model")
        loss_fun = ArcLoss()

    # Summary the model to find any thing suspicious at early stage.
    model.summary()

    # Construct an optimizer. This optimizer is different from the official
    # implementation which use SGD with momentum.
    optimizer = keras.optimizers.Adam(adam_alpha, amsgrad=True, epsilon=adam_epsilon)

    # Construct training datasets.
    dataset_train = build_dataset(train_files,
                                  batch_size=batch_size,
                                  one_hot_depth=num_ids,
                                  training=True,
                                  buffer_size=4096)

    # Construct dataset for validation. The loss value from this dataset can be
    # used to decide which checkpoint should be preserved.
    if val_files:
        dataset_val = build_dataset(val_files,
                                    batch_size=val_batch_size,
                                    one_hot_depth=val_num_ids,
                                    training=False,
                                    buffer_size=4096)

    # The training adventure is long and full of traps. A training supervisor
    # can help us to ease the pain.
    supervisor = TrainingSupervisor(model = model,
                                    optimizer=optimizer,
                                    loss=loss_fun,
                                    dataset=dataset_train,
                                    training_dir=training_dir,
                                    save_freq=frequency,
                                    monitor="categorical_accuracy",
                                    mode='max',
                                    name = name,
                                    val_freq = val_freq,
                                    val_dataset = dataset_val if val_files else None,
                                   )

    # If training accomplished, save the base model for inference.
    if export_only:
        print("The best model will be exported.")
        supervisor.restore(restore_weights_only, True)
        supervisor.export(base_model, export_dir)
        quit()

    # Restore the latest model if checkpoints are available.
    supervisor.restore(restore_weights_only)

    # Sometimes the training process might go wrong and we would like to resume
    # training from manually selected checkpoint. In this case some training
    # objects should be overridden before training started.
    if override:
        supervisor.override(0, 1)
        print("Training process overridden by user.")

    # Now it is safe to start training.
    supervisor.train(epochs, num_examples // batch_size, val_num_examples // val_batch_size if val_files else None)

    # Export the model after training.
    supervisor.export(base_model, export_dir)


In [ ]:
name='processed_merge'

input_shape = (128, 128, 3)
embedding_size = 512
num_ids = 1142
num_examples = 24721
val_num_ids = 1142
val_num_examples= 6181
val_batch_size = 16
batch_size = 16
epochs = 12

train_files = 'datasets/'+name+'_train.record'
val_files = 'datasets/'+name+'_val.record'
checkpoint_dir = 'outputs/exported/'+ name +'_soft/'

regularizer = keras.regularizers.L2(5e-4)
base_model = resnet101(input_shape=input_shape, output_size=embedding_size,
                           trainable=True,
                           training=True,
                           kernel_regularizer=regularizer,
                           name="embedding_model")

train(base_model, name = name+"_soft", train_files = train_files, test_files = None, val_files = val_files, input_shape = input_shape,
          num_ids = num_ids, num_examples = num_examples, training_dir = 'outputs/', val_num_ids = val_num_ids, val_num_examples = val_num_examples,
          val_batch_size = val_batch_size, val_freq = 5, 
          frequency = 10000, softmax = True, adam_alpha=0.001, adam_epsilon=0.001, batch_size = batch_size, export_only = False,
          override = False, epochs = epochs, restore_weights_only=True)

Building training model with softmax loss...
Model: "training_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_model (Functional) (None, 512)               43685888  
_________________________________________________________________
dense_3 (Dense)              (None, 1142)              585846    
_________________________________________________________________
softmax (Softmax)            (None, 1142)              0         
Total params: 44,271,734
Trainable params: 44,168,950
Non-trainable params: 102,784
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this 

2022-07-27 20:57:57.982306: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-27 20:57:57.983128: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3191995000 Hz


Restoring..
Only the model weights will be restored.
Checkpoint restored: None
Resume training from global step: 0, epoch: 1
Current step is: 0

Epoch 1/120


  0%|-------------------------------------------------------------------------------------------------| 0/1545 [00:00<?, ?it/s]2022-07-27 20:58:05.629363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-27 20:58:05.949889: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-07-27 20:58:07.089458: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-07-27 20:58:07.132675: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 1545/1545 [04:15<00:00,  6.06it/s, loss=137.71, accuracy=0.027]


Checkpoint saved at global step: 9270, to file: outputs/checkpoints/processed_merge_soft/ckpt-8

Epoch 7/120


 47%|>>>>>>>>>>>>>>>>>>>>>>>>>>>-------------------------------| 730/1545 [01:58<02:12,  6.16it/s, loss=141.19, accuracy=0.028]

Training accuracy: 0.0280, mean loss: 62.91
Monitor value not improved: 0.0339, latest: 0.0280.
Checkpoint saved at global step: 10000, to file: outputs/checkpoints/processed_merge_soft/ckpt-9


 97%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>--| 1501/1545 [04:07<00:07,  6.18it/s, loss=162.64, accuracy=0.026]

# Load softmax model and train with softmax = False

In [5]:
name='processed_merge'

input_shape = (128, 128, 3)
embedding_size = 512
num_ids = 1142
num_examples = 24721
val_num_ids = 1142
val_num_examples= 6181
val_batch_size = 32
batch_size = 32
epochs = 120

train_files = 'datasets/'+name+'_train.record'
val_files = 'datasets/'+name+'_val.record'
checkpoint_dir = 'outputs/exported/'+ name +'_soft/'

regularizer = keras.regularizers.L2(5e-4)
base_model = keras.models.load_model(checkpoint_dir)


train(base_model, name = name+"_arc", train_files = train_files, test_files = None, val_files = val_files, input_shape = input_shape,
          num_ids = num_ids, num_examples = num_examples, training_dir = 'outputs/', val_num_ids = val_num_ids, val_num_examples = val_num_examples,
          val_batch_size = val_batch_size, val_freq = 5,
          frequency = 10000, softmax = False, adam_alpha=0.001, adam_epsilon=0.001, batch_size = batch_size, export_only = False,
          override = False, epochs = epochs, restore_weights_only=True)


Building training model with ARC loss...
Model: "training_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 128, 128, 3)       0         
                                                                 
 embedding_model (Functional  (None, 512)              43685888  
 )                                                               
                                                                 
 l2_normalization (L2Normali  (None, 512)              0         
 zation)                                                         
                                                                 
 arc_layer (ArcLayer)        (None, 1142)              584704    
                                                                 
Total params: 44,270,592
Trainable params: 44,167,808
Non-trainable params: 102,784
___________________________________________________________

100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 135/135 [00:33<00:00,  4.45it/s, loss=2.02, accuracy=0.032]

Training accuracy: 0.0321, mean loss: 16.05
Monitor value improved from 0.0000 to 0.0321.
Best model found and saved: outputs/model_scout/processed_merge_arc/ckpt-1


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 135/135 [00:35<00:00,  3.82it/s, loss=2.02, accuracy=0.032]


Checkpoint saved at global step: 135, to file: outputs/checkpoints/processed_merge_arc/ckpt-2

Epoch 2/20


...


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 56/56 [00:04<00:00, 13.92it/s, loss=0.15, accuracy=0.025]


Training accuracy: 0.0248, mean loss: 10.29
Monitor value not improved: 0.0340, latest: 0.0248.


100%|>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 135/135 [00:36<00:00,  3.70it/s, loss=0.94, accuracy=0.025]

Checkpoint saved at global step: 2700, to file: outputs/checkpoints/processed_merge_arc/ckpt-24
Training accomplished at epoch 20
Saving model to outputs/exported/processed_merge_arc ...


INFO:tensorflow:Assets written to: outputs/exported/processed_merge_arc/assets
Model saved at: outputs/exported/processed_merge_arc


# Classifier

In [14]:
def save_pkl(pkl, path = 'model.pkl'):
    with open(path, 'wb') as f:
        pickle.dump(pkl, f)
    print("saved pkl file at:",path)

def load_pkl(path='model.pkl'):
    with open(path, 'rb') as f:
        pkl = pickle.load(f)
    return pkl

def findCosineDistance(vector1, vector2):
    """
    Calculate cosine distance between two vector
    """
    vec1 = vector1.flatten()
    vec2 = vector2.flatten()

    a = np.dot(vec1.T, vec2)
    b = np.dot(vec1.T, vec1)
    c = np.dot(vec2.T, vec2)
    return 1 - (a/(np.sqrt(b)*np.sqrt(c)))

def cosine_similarity(test_vec, source_vecs):
    """
    Verify the similarity of one vector to group vectors of one class
    """
    cos_dist = 0
    for source_vec in source_vecs:
        cos_dist += findCosineDistance(test_vec, source_vec)
    return cos_dist/len(source_vecs)

def make_embeddings(dataset_path='datasets/sorted_palmvein_roi/', output_path='outputs/', model_dir='outputs/exported/arcface', types='bmp'):
    # Grab the paths to the input images in our dataset
    print("[INFO] quantifying palms...")
    # imagePaths =[]
    # [imagePaths.extend(glob(dataset_path+"*/*."+typ)) for typ in types]
    
    samples =[]
    [samples.extend(glob(dataset_path+"*/*."+typ)) for typ in types]
    labels = [image_path.split('/')[-2] for image_path in samples]
    sample_labels = [image_path.split('/')[-2] for image_path in samples]
    samples_train, imagePaths = train_test_split(samples, test_size=0.2, random_state=42, stratify=sample_labels)
    
#     imagePaths.extend(glob(dataset_path+'/*/*.jpg'))

    # Initialize model
    embedding_model = keras.models.load_model(model_dir)
       
    # Initialize our lists of extracted facial embeddings and corresponding people names
    knownEmbeddings = []
    knownNames = []

    # Initialize the total number of faces processed
    total = 0

    # Loop over the imagePaths
    for (i, imagePath) in tqdm(enumerate(imagePaths)):
        # extract the person name from the image path
        name = imagePath.split(os.path.sep)[-2]
#         print(imagePath)

         # load the image
        img = cv2.imread(imagePath).reshape(-1,128,128,3)
        palms_embedding = embedding_model.predict(img)[0]
        # add the name of the person + corresponding face
        # embedding to their respective list
        knownNames.append(name)
        knownEmbeddings.append(palms_embedding)
        total += 1
        
    print(total, " palms embedded")
    print(f"number of classes: {len(set(knownNames))}")

    # save to output
    data = {"embeddings": knownEmbeddings, "names": knownNames}
    save_pkl(pkl=data, path=output_path+'db.pkl')
    
def make_model(embeddings_path='outputs/db.pkl', output_path='outputs/'):
    # Load the face embeddings
    data = load_pkl(embeddings_path)
    num_classes = len(np.unique(data["names"]))
    y = np.array(data["names"])
    X = np.array(data["embeddings"])
    
    
    # Initialize Softmax training model arguments
    input_shape = X.shape[1]
    cv = KFold(n_splits=5, random_state=1, shuffle=True)

    model =  MLPClassifier(hidden_layer_sizes=(input_shape, 640, 112, 640, num_classes), activation='tanh',max_iter=100, batch_size='auto', learning_rate='adaptive',
                           validation_fraction=0.0, solver='adam', early_stopping=False ,verbose=0,random_state=1)

    for train_idx, valid_idx in cv.split(X):
        model.fit(X[train_idx], y[train_idx],)
        print(model.score(X[valid_idx], y[valid_idx]), end='\t')
    
    # save_pkl(pkl=model, path=output_path+'model.pkl')
    
    return model

In [15]:
name="processed_resnet"
make_embeddings(dataset_path='datasets/processed_merge/', output_path='outputs/exported/'+name+'_arc/', model_dir='outputs/exported/'+name+'_arc/',types=['png','jpg','jepg','png', 'bmp'])

[INFO] quantifying palms...


6181it [07:17, 14.14it/s]

6181  palms embedded
number of classes: 1142
saved pkl file at: outputs/exported/processed_resnet_arc/db.pkl


In [ ]:
make_model(embeddings_path='outputs/exported/'+name+'_arc/db.pkl', output_path='outputs/exported/'+name+'_arc/')

0.30072756669361356	0.32119741100323623	0.30906148867313915	

In [10]:
embedding_model = keras.models.load_model('outputs/exported/'+name+'_arc/')
model = load_pkl('outputs/exported/'+name+'_arc/model.pkl')
samples = glob('datasets/processed_merge/*/*.jpg')

In [13]:
img_paths = samples[0:20]
imgs = [cv2.imread(img).reshape(-1, 128, 128, 3) for img in img_paths]
embedding = [embedding_model.predict(img)[0] for img in imgs]

[print(np.format_float_positional((cosine_similarity(i, embedding[:10])), precision=3), end='\t') for i in embedding]
print()
[print(i, end='\t') for i in model.predict(embedding)]
print()
[print(img.split("/")[-2], end='\t') for img in img_paths] 
print()

0.583	0.456	0.488	0.505	0.483	0.499	0.549	0.480	0.48	0.575	0.815	0.704	0.680	0.775	0.830	0.731	0.787	0.812	0.8	0.783	
milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	
milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	milad_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	Dr.cheshomi_L	


In [ ]:
embedding_model.save(